# Scikit-learn API

For those not familiar with PyTorch, we've created a wrapper for scikit-learn. This contains the familiar fit/predict-methods.

In [1]:
from binn import BINNClassifier, Network, SuperLogger
import pandas as pd


/home/erikh/BINN/BINN/test-venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Similar to the PyTorch API, we load data and create a network, however, now we instead create a BINNClassifier object (this is the scikit-learn wrapper class).

In [2]:
pathways = pd.read_csv("../data/pathways.tsv", sep="\t")
translation = pd.read_csv("../data/translation.tsv", sep="\t")
input_data = pd.read_csv("../data/test_qm.csv")
design_matrix = pd.read_csv("../data/design_matrix.tsv", sep="\t")

network = Network(
    input_data=input_data,
    pathways=pathways,
    mapping=translation,
    source_column="child",
    target_column="parent"
)

binn = BINNClassifier(
    network=network,
    n_layers=4,
    dropout=0.2,
    epochs=3,
    threads=10,
    logger=SuperLogger("logs/test")
)
binn.clf.features

Index(['A0M8Q6', 'O00194', 'O00391', 'O14786', 'O14791', 'O15145', 'O43707',
       'O75369', 'O75594', 'O75636',
       ...
       'Q9UBE0', 'Q9UBQ7', 'Q9UBR2', 'Q9UBX5', 'Q9UGM3', 'Q9UK55', 'Q9UNW1',
       'Q9Y490', 'Q9Y4L1', 'Q9Y6Z7'],
      dtype='object', length=449)

We have to make our data-matrix fit the input layer in the BINN. Then we fit the BINN.

In [3]:
from util_for_examples import generate_data, fit_data_matrix_to_network_input

X = fit_data_matrix_to_network_input(input_data.reset_index(), features=binn.clf.features)

X, y = generate_data(X, design_matrix)

X_test = X[:10]
X_train = X[10:]
y_test = y[:10]
y_train = y[10:]

binn.fit(X_train, y_train, epochs=5)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name     | Type             | Params
----------------------------------------------
0 | layers   | Sequential       | 364 K 
1 | loss     | CrossEntropyLoss | 0     
2 | loss_val | CrossEntropyLoss | 0     
----------------------------------------------
364 K     Trainable params
0         Non-trainable params
364 K     Total params
1.457     Total estimated model params size (MB)
Experiment logs directory logs/test/lightning_logs/version_3 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
The number of training batches (24) is smaller than the logging interval Trainer(log_every_n_steps=50)

Epoch 4: 100%|██████████| 24/24 [00:00<00:00, 57.06it/s, loss=0.75, v_num=3, train_loss=0.773, train_acc=0.556] 

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 24/24 [00:09<00:00,  2.66it/s, loss=0.75, v_num=3, train_loss=0.773, train_acc=0.556]


We can predict some instances.

In [4]:
binn.predict(X_test)

tensor([[-0.7593, -0.6118],
        [ 0.1700, -0.8432],
        [ 0.0709, -1.0857],
        [-0.2082, -1.1145],
        [ 0.1687, -1.3955],
        [-0.7921,  0.4828],
        [ 0.0444, -0.1737],
        [-1.5466,  0.4932],
        [-0.5567,  1.1309],
        [ 0.2844,  0.0567]])